In [1]:
import leafmap
import geopandas as gpd
from geospatial_tools import DATA_DIR
from geospatial_tools.stac import StacSearch
from geospatial_tools.utils import get_yaml_config, download_url, unzip_file


## Base data

The USA polygon is base off 2018's `cb_2018_us_nation_5m` shapefile, taken from here: 
https://www.census.gov/geographies/mapping-files/time-series/geo/carto-boundary-file.html

It was then processed using QGIS to keep only the contiguous states, without any islands.

The Sentinel 2 grid was taken from the kml file found here: 
https://sentiwiki.copernicus.eu/web/s2-products

Below is some code to help with the download part.

### Downloading data
Let's download our source data

In [2]:
file_configs = get_yaml_config("data_file_links")
raw_usa_polygon_path = file_configs["united_states_polygon"]["url"]
raw_s2_tiling_grid_path = file_configs["sentinel_2_tiling_grid"]["url"]
download_list = {"raw_usa_polygon": raw_usa_polygon_path, "raw_s2_tiling_grid": raw_s2_tiling_grid_path}
file_list = [download_url(url=url, filename=f"{DATA_DIR}/{key}.zip") for key, url in download_list.items()]

file_list



[2024-08-23 11:01:45] INFO       [MainThread][geospatial_tools.utils] Yaml config file [/home/francispelletier/projects/geospatial_tools/configs/data_file_links.yaml] found.
[2024-08-23 11:01:45] INFO       [MainThread][geospatial_tools.utils] Loading YAML config file [/home/francispelletier/projects/geospatial_tools/configs/data_file_links.yaml].
[2024-08-23 11:01:46] INFO       [MainThread][geospatial_tools.utils] Downloaded /home/francispelletier/projects/geospatial_tools/data/raw_usa_polygon.zip successfully.
[2024-08-23 11:01:47] INFO       [MainThread][geospatial_tools.utils] Downloaded /home/francispelletier/projects/geospatial_tools/data/raw_s2_tiling_grid.zip successfully.


['/home/francispelletier/projects/geospatial_tools/data/raw_usa_polygon.zip',
 '/home/francispelletier/projects/geospatial_tools/data/raw_s2_tiling_grid.zip']

In [3]:
[unzip_file(zip_path=f, extract_to=DATA_DIR) for f in file_list]

[2024-08-23 11:01:47] INFO       [MainThread][geospatial_tools.utils] Extracted: [cb_2018_us_nation_20m.shp.ea.iso.xml]
[2024-08-23 11:01:47] INFO       [MainThread][geospatial_tools.utils] Extracted: [cb_2018_us_nation_20m.shp.iso.xml]
[2024-08-23 11:01:47] INFO       [MainThread][geospatial_tools.utils] Extracted: [cb_2018_us_nation_20m.shp]
[2024-08-23 11:01:47] INFO       [MainThread][geospatial_tools.utils] Extracted: [cb_2018_us_nation_20m.shx]
[2024-08-23 11:01:47] INFO       [MainThread][geospatial_tools.utils] Extracted: [cb_2018_us_nation_20m.dbf]
[2024-08-23 11:01:47] INFO       [MainThread][geospatial_tools.utils] Extracted: [cb_2018_us_nation_20m.prj]
[2024-08-23 11:01:47] INFO       [MainThread][geospatial_tools.utils] Extracted: [cb_2018_us_nation_20m.cpg]
[2024-08-23 11:01:47] INFO       [MainThread][geospatial_tools.utils] Extracted: [S2A_OPER_GIP_TILPAR_MPC__20151209T095117_V20150622T000000_21000101T000000_B00.kml]


[None, None]

### Initial pre-processing

The above layers were processed using QGIS.

For the purpose of this analysis, only the contiguous lower 48 states have been conserved; smaller islands/land masses 
have also been striped.

The S2 tiling grid has been trimmed to keep only the grid cells that overlap with the 
contiguous states.

Since our area of study is quite large, the `EPSG:5070` projection was chosen, as it
covers the whole area, introduces minimal distortion while preserving area.

The files below have also been saved in this repository.

In [4]:
USA_POLYGON_FILE = DATA_DIR / "usa_polygon_5070.gpkg"
S2_USA_GRID_FILE = DATA_DIR / "s2_grid_usa_polygon_5070.gpkg"
usa_polygon = gpd.read_file(USA_POLYGON_FILE)
s2_grid = gpd.read_file(S2_USA_GRID_FILE)

In [5]:
usa_polygon

,AFFGEOID,GEOID,NAME,geometry
0,0100000US,US,United States,"MULTIPOLYGON (((-2116048.733 3142966.552, -211..."


In [6]:
s2_grid

,name,folders,description,geometry
0,12TUP,Features,TILE PROPERTIES<br><table border=0 cellpadding...,MULTIPOLYGON Z (((-1386334.944 2487548.770 0.0...
1,12TYQ,Features,TILE PROPERTIES<br><table border=0 cellpadding...,MULTIPOLYGON Z (((-976300.478 2523767.452 0.00...
2,12TYR,Features,TILE PROPERTIES<br><table border=0 cellpadding...,MULTIPOLYGON Z (((-960099.705 2622374.255 0.00...
3,12TYN,Features,TILE PROPERTIES<br><table border=0 cellpadding...,MULTIPOLYGON Z (((-1008622.024 2325748.358 0.0...
4,12TYP,Features,TILE PROPERTIES<br><table border=0 cellpadding...,MULTIPOLYGON Z (((-992478.385 2424861.340 0.00...
...,...,...,...,...
977,12TTM,Features,TILE PROPERTIES<br><table border=0 cellpadding...,MULTIPOLYGON Z (((-1515431.586 2304192.826 0.0...
978,12TUK,Features,TILE PROPERTIES<br><table border=0 cellpadding...,MULTIPOLYGON Z (((-1448525.813 2089886.667 0.0...
979,12TUQ,Features,TILE PROPERTIES<br><table border=0 cellpadding...,MULTIPOLYGON Z (((-1371006.917 2586590.133 0.0...
980,12TUR,Features,TILE PROPERTIES<br><table border=0 cellpadding...,MULTIPOLYGON Z (((-1355793.563 2685354.080 0.0...


In [7]:
m = leafmap.Map(center=[40, -98], zoom=4)

# In blue, the USA polygon
m.add_gdf(usa_polygon, layer_name='usa')
# In red, the Sentinel 2 grid
m.add_gdf(s2_grid, layer_name='s2_grid', style={"color": "red"})

m

Map(center=[40, -98], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title', 'zoom_out_te…

## Creating our grid

From this, we want to create a grid of square polygons with which we will later on
query the [Planetary Computer](https://planetarycomputer.microsoft.com/dataset/sentinel-2-l2a)
Sentinel 2 dataset and clip the selected Sentinel 2 images.

For the purpose of this notebook, the grid that will be created will use 10km by 10km squares to speed up 
processing.

In [8]:
from geospatial_tools.vector import create_vector_grid_parallel, to_geopackage, select_polygons_by_location

In [9]:
grid_size = 5000
bbox = usa_polygon.total_bounds
grid_5km_filename = DATA_DIR / "polygon_grid_5km.gpkg"

In [10]:
print("Starting processing for [create_vector_grid_parallel]")
grid_5km = create_vector_grid_parallel(bounding_box=bbox, grid_size=grid_size, crs="EPSG:5070")
print(f"Printing len(grid_parallel) to check if grid contains same amount of polygons : {len(grid_5km)}")
to_geopackage(gdf=grid_5km, filename=grid_5km_filename)
grid_5km

Starting processing for [create_vector_grid_parallel]
[2024-08-23 11:01:48] INFO       [MainThread][geospatial_tools.vector] Creating grid coordinates for bounding box [[-2356113.74289801   301469.31619713  2258154.44089948  3165721.6501298 ]]
[2024-08-23 11:01:48] INFO       [MainThread][geospatial_tools.vector] Creating flattened grid coordinates
[2024-08-23 11:01:48] INFO       [MainThread][geospatial_tools.vector] Number of workers used: 16
[2024-08-23 11:01:48] INFO       [MainThread][geospatial_tools.vector] Allocating polygon array for [528879] polygons
[2024-08-23 11:01:48] INFO       [MainThread][geospatial_tools.vector] Creating polygons from chunks
Printing len(grid_parallel) to check if grid contains same amount of polygons : 528879
[2024-08-23 11:01:56] INFO       [MainThread][geospatial_tools.vector] Starting writing process
[2024-08-23 11:01:58] INFO       [MainThread][geospatial_tools.vector] File [/home/francispelletier/projects/geospatial_tools/data/polygon_grid_5km.g

,geometry,feature_id
0,"POLYGON ((-2356113.743 301469.316, -2351113.74...",5aab9102-242b-4df4-a91c-e014a62b8484
1,"POLYGON ((-2351113.743 301469.316, -2346113.74...",abbd71a3-6efc-4ae6-a51e-80c6df2fa5c9
2,"POLYGON ((-2346113.743 301469.316, -2341113.74...",be453a43-c354-42a6-989d-151d6efe897a
3,"POLYGON ((-2341113.743 301469.316, -2336113.74...",1343fbe8-ca51-4319-98e6-a844e92fd862
4,"POLYGON ((-2336113.743 301469.316, -2331113.74...",bfa792f5-c5c0-498e-86d2-2edf240cb16b
...,...,...
528874,"POLYGON ((2233886.257 3161469.316, 2238886.257...",0502b15d-b1c0-4f46-8211-c804354b980f
528875,"POLYGON ((2238886.257 3161469.316, 2243886.257...",679cbc3f-098a-44a0-b686-7e136ca1ff02
528876,"POLYGON ((2243886.257 3161469.316, 2248886.257...",b2ebe0e2-a0ae-4cad-910f-e8af9cd812a7
528877,"POLYGON ((2248886.257 3161469.316, 2253886.257...",d94dcc98-0cc1-49a9-8e54-9987f8332bd3


### Selecting the useful polygons

Now, since our grid was created using the extent of our input polygon (continental USA), we need to filter out the polygons that do not intersect with it.

Doing this in Python is not the most efficient way to do things, but since it's a step that shouldn't be done over and over, it's not that critical.

If ever you need to do this step in an efficient way because the data is just too big or too complex, it would be better off going through QGIS, PyGQIS, GDAL or 
some other more efficient way to do this operation. 

In [11]:
intersecting_polygons_filename = DATA_DIR / "intersecting_polygon_grid_5km.gpkg"
print("Starting intersect selection")
intersecting_polygons = select_polygons_by_location(grid_5km, usa_polygon)

# Optionally save to a new file
to_geopackage(intersecting_polygons, intersecting_polygons_filename)
intersecting_polygons

Starting intersect selection
[2024-08-23 11:01:58] INFO       [MainThread][geospatial_tools.vector] Number of workers used: 16
[2024-08-23 11:02:18] INFO       [MainThread][geospatial_tools.vector] Starting writing process
[2024-08-23 11:02:19] INFO       [MainThread][geospatial_tools.vector] File [/home/francispelletier/projects/geospatial_tools/data/intersecting_polygon_grid_5km.gpkg] took 1.5032312870025635 seconds to write.


,geometry,feature_id
0,"POLYGON ((1513886.257 301469.316, 1518886.257 ...",92658cf3-01e5-449e-bd3a-36201c960e11
1,"POLYGON ((1518886.257 301469.316, 1523886.257 ...",239ecae1-64da-4a0b-a765-bfbc0af4956a
2,"POLYGON ((1523886.257 301469.316, 1528886.257 ...",d2890617-f83a-4461-be0e-0348c31bab82
3,"POLYGON ((1528886.257 301469.316, 1533886.257 ...",65829167-1b91-41d3-ae69-aefb2014749a
4,"POLYGON ((-146113.743 306469.316, -141113.743 ...",d27589a1-95bf-49b3-8b6e-f102121d1ac8
...,...,...
316126,"POLYGON ((-1966113.743 3161469.316, -1961113.7...",0c75d021-de17-4e55-b18f-6f24c1807872
316127,"POLYGON ((-1961113.743 3161469.316, -1956113.7...",00896fa6-1c61-4810-bf62-c42a6f190f83
316128,"POLYGON ((-1956113.743 3161469.316, -1951113.7...",f377e3be-6a9e-4f78-ac3a-a9b6387f4e72
316129,"POLYGON ((-1951113.743 3161469.316, -1946113.7...",11c136f9-84d9-4286-a6c4-423e1fa21203


### Visualizing the selected polygons

This will take more or less time, depending on the number on polygons.

(Do not try this with a grid size smaller than 10000m)

In [12]:
# This takes a few minutes and navigation will be slow
# m_intersecting_polygons = leafmap.Map(center=[40, -98], zoom=4)
# m_intersecting_polygons.add_gdf(intersecting_polygons, layer_name='intersecting_polygons', style={"color": "blue"})
# m_intersecting_polygons

## Making of list of all the S2 tiling grids we will be using

Now, we need to build our S2 tile grid list

In [13]:
s2_tile_grid_list = s2_grid["name"].to_list()
s2_tile_grid_list

['12TUP',
 '12TYQ',
 '12TYR',
 '12TYN',
 '12TYP',
 '12TYS',
 '12TYT',
 '11SMB',
 '11SMC',
 '11SLV',
 '11SMA',
 '11SMS',
 '12UUV',
 '11SMT',
 '11SMD',
 '11SMR',
 '12UUU',
 '11SNA',
 '12TWS',
 '11SNB',
 '12TWT',
 '11SMU',
 '12TWQ',
 '11SMV',
 '12TWR',
 '12TXM',
 '11SNS',
 '12TXN',
 '11SNC',
 '12TXK',
 '11SND',
 '12TXL',
 '11SKD',
 '12TXR',
 '12TXS',
 '11SKB',
 '12TXP',
 '11SKC',
 '12TXQ',
 '11SKU',
 '12TYL',
 '11SKV',
 '12TYM',
 '12TXT',
 '11SKT',
 '12TYK',
 '11SLC',
 '11SLD',
 '11SLA',
 '11SLB',
 '11SLT',
 '11SLU',
 '19TBF',
 '19TBG',
 '19TDM',
 '19TDN',
 '19TEJ',
 '19TEK',
 '19TEN',
 '19TEL',
 '19TEM',
 '18STE',
 '18STF',
 '19TCG',
 '18STC',
 '19TCH',
 '18STD',
 '18STJ',
 '19TCF',
 '19TCL',
 '18STG',
 '19TCM',
 '18STH',
 '19TCJ',
 '19TCK',
 '19TDF',
 '19TDG',
 '19TDK',
 '19TDL',
 '19TDJ',
 '16TCQ',
 '16TCR',
 '16TCN',
 '16TCP',
 '16TDK',
 '16TDL',
 '16TCS',
 '16TCT',
 '16TDP',
 '15RWQ',
 '16TDQ',
 '16TDM',
 '15RXN',
 '16TDN',
 '15RXP',
 '16TDT',
 '16TEK',
 '16TDR',
 '16TDS',
 '16SGB',


# Finding the best image for each S2 tiling grid

### Subselection
This is a long list, so we'll be continuing this notebook with the following subset

In [14]:
s2_tile_grid_subset_list = ["10TDK", "10TEK", "10SDJ", "10SEJ"]

In [15]:
from geospatial_tools.utils import create_date_range_for_specific_period
from geospatial_tools.planetary_computer.sentinel_2 import find_best_product_per_s2_tile, write_results_to_file

In [16]:
s2_tile_grid_list = s2_tile_grid_subset_list

start_year = 2022
end_year = 2024
start_month = 6
end_month = 7

date_ranges_2020_2024 = create_date_range_for_specific_period(start_year=start_year, end_year=end_year,
                                                              start_month_range=start_month,
                                                              end_month_range=end_month)
max_cloud_cover = 15

tile_dictionary, error_list = find_best_product_per_s2_tile(date_ranges=date_ranges_2020_2024,
                                                            max_cloud_cover=max_cloud_cover,
                                                            s2_tile_grid_list=s2_tile_grid_list)

results_dict = write_results_to_file(cloud_cover=max_cloud_cover, tile_dictionary=tile_dictionary,
                                     error_list=error_list)
results_dict

[2024-08-23 11:02:20] INFO       [ThreadPoolExecutor-0_3][geospatial_tools.stac] Initiating STAC API search for the following date ranges : [['2022-06-01T00:00:00Z/2022-07-31T23:59:59Z', '2023-06-01T00:00:00Z/2023-07-31T23:59:59Z', '2024-06-01T00:00:00Z/2024-07-31T23:59:59Z'] 
	Query : [{'eo:cloud_cover': {'lt': 15}, 's2:mgrs_tile': {'in': ['10SEJ']}}]
[2024-08-23 11:02:20] INFO       [ThreadPoolExecutor-0_1][geospatial_tools.stac] Initiating STAC API search for the following date ranges : [['2022-06-01T00:00:00Z/2022-07-31T23:59:59Z', '2023-06-01T00:00:00Z/2023-07-31T23:59:59Z', '2024-06-01T00:00:00Z/2024-07-31T23:59:59Z'] 
	Query : [{'eo:cloud_cover': {'lt': 15}, 's2:mgrs_tile': {'in': ['10TEK']}}]
[2024-08-23 11:02:20] INFO       [ThreadPoolExecutor-0_0][geospatial_tools.stac] Initiating STAC API search for the following date ranges : [['2022-06-01T00:00:00Z/2022-07-31T23:59:59Z', '2023-06-01T00:00:00Z/2023-07-31T23:59:59Z', '2024-06-01T00:00:00Z/2024-07-31T23:59:59Z'] 
	Query : [{'

{'tile_filename': PosixPath('/home/francispelletier/projects/geospatial_tools/data/data_lt15cc.json'),
 'errors_filename': 'None'}

In [17]:
tile_dictionary

{'10TDK': {'id': 'S2A_MSIL2A_20240705T185921_R013_T10TDK_20240706T050412',
  'cloud_cover': 0.000707},
 '10TEK': {'id': 'S2A_MSIL2A_20220723T184931_R113_T10TEK_20220725T173028',
  'cloud_cover': 4e-06},
 '10SDJ': {'id': 'S2A_MSIL2A_20240712T184931_R113_T10SDJ_20240713T042744',
  'cloud_cover': 5.8e-05},
 '10SEJ': {'id': 'S2B_MSIL2A_20240730T185919_R013_T10SEJ_20240730T214651',
  'cloud_cover': 1.9e-05}}

## Data processing pipeline prototype

### Building our processing list

First, let's make create a subselection of our dataset

In [18]:
import geopandas as gpd
from geospatial_tools import DATA_DIR
from geospatial_tools.vector import spatial_join_within

In [19]:
S2_USA_GRID_FILE = DATA_DIR / "s2_grid_usa_polygon_5070.gpkg"
s2_grid = gpd.read_file(S2_USA_GRID_FILE)
s2_grid

,name,folders,description,geometry
0,12TUP,Features,TILE PROPERTIES<br><table border=0 cellpadding...,MULTIPOLYGON Z (((-1386334.944 2487548.770 0.0...
1,12TYQ,Features,TILE PROPERTIES<br><table border=0 cellpadding...,MULTIPOLYGON Z (((-976300.478 2523767.452 0.00...
2,12TYR,Features,TILE PROPERTIES<br><table border=0 cellpadding...,MULTIPOLYGON Z (((-960099.705 2622374.255 0.00...
3,12TYN,Features,TILE PROPERTIES<br><table border=0 cellpadding...,MULTIPOLYGON Z (((-1008622.024 2325748.358 0.0...
4,12TYP,Features,TILE PROPERTIES<br><table border=0 cellpadding...,MULTIPOLYGON Z (((-992478.385 2424861.340 0.00...
...,...,...,...,...
977,12TTM,Features,TILE PROPERTIES<br><table border=0 cellpadding...,MULTIPOLYGON Z (((-1515431.586 2304192.826 0.0...
978,12TUK,Features,TILE PROPERTIES<br><table border=0 cellpadding...,MULTIPOLYGON Z (((-1448525.813 2089886.667 0.0...
979,12TUQ,Features,TILE PROPERTIES<br><table border=0 cellpadding...,MULTIPOLYGON Z (((-1371006.917 2586590.133 0.0...
980,12TUR,Features,TILE PROPERTIES<br><table border=0 cellpadding...,MULTIPOLYGON Z (((-1355793.563 2685354.080 0.0...


In [20]:
S2_USA_GRID_SUBSET_FILE = DATA_DIR / "s2_grid_usa_polygon_5070_subset.gpkg"
s2_grid_subset = s2_grid[s2_grid["name"].isin(s2_tile_grid_subset_list)]

# Optionally save to geopackage
to_geopackage(gdf=s2_grid_subset, filename=S2_USA_GRID_SUBSET_FILE)

s2_grid_subset

[2024-08-23 11:02:23] INFO       [MainThread][geospatial_tools.vector] Starting writing process
[2024-08-23 11:02:23] INFO       [MainThread][geospatial_tools.vector] File [/home/francispelletier/projects/geospatial_tools/data/s2_grid_usa_polygon_5070_subset.gpkg] took 0.012789726257324219 seconds to write.


,name,folders,description,geometry
823,10SEJ,Features,TILE PROPERTIES<br><table border=0 cellpadding...,MULTIPOLYGON Z (((-2262059.689 2182514.584 0.0...
841,10SDJ,Features,TILE PROPERTIES<br><table border=0 cellpadding...,MULTIPOLYGON Z (((-2357236.175 2210256.163 0.0...
846,10TDK,Features,TILE PROPERTIES<br><table border=0 cellpadding...,MULTIPOLYGON Z (((-2329019.777 2307113.875 0.0...
857,10TEK,Features,TILE PROPERTIES<br><table border=0 cellpadding...,MULTIPOLYGON Z (((-2233778.019 2279366.081 0.0...


In [21]:
intersect_polygons_subset_filename = DATA_DIR / "intersecting_polygon_grid_5km_subset.gpkg"
print("Starting intersect selection")
intersect_polygons_subset = select_polygons_by_location(intersecting_polygons, s2_grid_subset, predicate="within")

# Optionally save to a new file
to_geopackage(intersect_polygons_subset, intersect_polygons_subset_filename)
intersect_polygons_subset

Starting intersect selection
[2024-08-23 11:02:23] INFO       [MainThread][geospatial_tools.vector] Number of workers used: 16
[2024-08-23 11:02:24] INFO       [MainThread][geospatial_tools.vector] Starting writing process
[2024-08-23 11:02:24] INFO       [MainThread][geospatial_tools.vector] File [/home/francispelletier/projects/geospatial_tools/data/intersecting_polygon_grid_5km_subset.gpkg] took 0.02536630630493164 seconds to write.


,geometry,feature_id
0,"POLYGON ((-2206113.743 2051469.316, -2201113.7...",76f33694-bbf2-4def-b1dc-6a891fc9877c
1,"POLYGON ((-2201113.743 2051469.316, -2196113.7...",fc04cdff-07ae-4744-90e5-90c3da9e4451
2,"POLYGON ((-2196113.743 2051469.316, -2191113.7...",74e17840-16a3-4c4e-84e8-25a2cffa19a1
3,"POLYGON ((-2221113.743 2056469.316, -2216113.7...",4287d16f-a641-4392-80bd-aad968f74125
4,"POLYGON ((-2216113.743 2056469.316, -2211113.7...",b2ece92e-06ca-4ae8-b612-9d72f3e05dbe
...,...,...
1571,"POLYGON ((-2306113.743 2291469.316, -2301113.7...",97ac8454-3728-4af4-a7b8-30e6cbfe881c
1572,"POLYGON ((-2301113.743 2291469.316, -2296113.7...",f951fb10-5825-4f51-9c84-586324d7cdf8
1573,"POLYGON ((-2326113.743 2296469.316, -2321113.7...",e8c0536a-597c-4c6d-9554-0314d425e35b
1574,"POLYGON ((-2321113.743 2296469.316, -2316113.7...",76d83ac8-2b32-479e-a511-fae8d9dc78fb


In [22]:
m_subset = leafmap.Map(center=[39.7, -123], zoom=8)
m_subset.add_gdf(s2_grid_subset, layer_name='s2_tiles', style={"color": "red"})
m_subset.add_gdf(intersect_polygons_subset, layer_name='intersecting_polygons', style={"color": "blue"})
m_subset

Map(center=[39.7, -123], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title', 'zoom_out…

In [23]:
from geospatial_tools.planetary_computer.sentinel_2 import BestProductsForFeatures

In [24]:
s2_feature_name_columns = "name"
vector_column_name = "s2_tiles"

best_products_client = BestProductsForFeatures(sentinel2_tiling_grid=s2_grid_subset,
                                               sentinel2_tiling_grid_column=s2_feature_name_columns,
                                               vector_features=intersect_polygons_subset,
                                               vector_features_column=vector_column_name,
                                               max_cloud_cover=15)

In [25]:
start_year = 2022
end_year = 2024
start_month = 6
end_month = 7

best_products_client.create_date_range(start_year, end_year, start_month, end_month)

products = best_products_client.find_best_products()
products

[2024-08-23 11:02:26] INFO       [ThreadPoolExecutor-1_0][geospatial_tools.stac] Initiating STAC API search for the following date ranges : [['2022-06-01T00:00:00Z/2022-07-31T23:59:59Z', '2023-06-01T00:00:00Z/2023-07-31T23:59:59Z', '2024-06-01T00:00:00Z/2024-07-31T23:59:59Z'] 
	Query : [{'eo:cloud_cover': {'lt': 15}, 's2:mgrs_tile': {'in': ['10SEJ']}}]
[2024-08-23 11:02:26] INFO       [ThreadPoolExecutor-1_1][geospatial_tools.stac] Initiating STAC API search for the following date ranges : [['2022-06-01T00:00:00Z/2022-07-31T23:59:59Z', '2023-06-01T00:00:00Z/2023-07-31T23:59:59Z', '2024-06-01T00:00:00Z/2024-07-31T23:59:59Z'] 
	Query : [{'eo:cloud_cover': {'lt': 15}, 's2:mgrs_tile': {'in': ['10SDJ']}}]
[2024-08-23 11:02:26] INFO       [ThreadPoolExecutor-1_2][geospatial_tools.stac] Initiating STAC API search for the following date ranges : [['2022-06-01T00:00:00Z/2022-07-31T23:59:59Z', '2023-06-01T00:00:00Z/2023-07-31T23:59:59Z', '2024-06-01T00:00:00Z/2024-07-31T23:59:59Z'] 
	Query : [{'

{'10SEJ': {'id': 'S2B_MSIL2A_20240730T185919_R013_T10SEJ_20240730T214651',
  'cloud_cover': 1.9e-05},
 '10SDJ': {'id': 'S2A_MSIL2A_20240712T184931_R113_T10SDJ_20240713T042744',
  'cloud_cover': 5.8e-05},
 '10TDK': {'id': 'S2A_MSIL2A_20240705T185921_R013_T10TDK_20240706T050412',
  'cloud_cover': 0.000707},
 '10TEK': {'id': 'S2A_MSIL2A_20220723T184931_R113_T10TEK_20220725T173028',
  'cloud_cover': 4e-06}}

In [26]:
best_results = best_products_client.select_best_products_per_feature()
to_geopackage(best_results, DATA_DIR / "vector_tiles_with_s2tiles_subset.gpkg")
best_results


[2024-08-23 11:02:56] INFO       [MainThread][geospatial_tools.vector] Creating temporary UUID field for join operations
[2024-08-23 11:02:56] INFO       [MainThread][geospatial_tools.vector] Starting process to find and identify contained features using spatial 'within' join operation
[2024-08-23 11:02:56] INFO       [MainThread][geospatial_tools.vector] Grouping results
[2024-08-23 11:02:56] INFO       [MainThread][geospatial_tools.vector] Cleaning and merging results
[2024-08-23 11:02:56] INFO       [MainThread][geospatial_tools.vector] Spatial join operation is completed
s2_tiles_search_results: {'10SEJ': {'id': 'S2B_MSIL2A_20240730T185919_R013_T10SEJ_20240730T214651', 'cloud_cover': 1.9e-05}, '10SDJ': {'id': 'S2A_MSIL2A_20240712T184931_R113_T10SDJ_20240713T042744', 'cloud_cover': 5.8e-05}, '10TDK': {'id': 'S2A_MSIL2A_20240705T185921_R013_T10TDK_20240706T050412', 'cloud_cover': 0.000707}, '10TEK': {'id': 'S2A_MSIL2A_20220723T184931_R113_T10TEK_20220725T173028', 'cloud_cover': 4e-06

,geometry,feature_id,s2_tiles,best_s2_product_id
0,"POLYGON ((-2206113.743 2051469.316, -2201113.7...",76f33694-bbf2-4def-b1dc-6a891fc9877c,[10SEJ],S2B_MSIL2A_20240730T185919_R013_T10SEJ_2024073...
1,"POLYGON ((-2201113.743 2051469.316, -2196113.7...",fc04cdff-07ae-4744-90e5-90c3da9e4451,[10SEJ],S2B_MSIL2A_20240730T185919_R013_T10SEJ_2024073...
2,"POLYGON ((-2196113.743 2051469.316, -2191113.7...",74e17840-16a3-4c4e-84e8-25a2cffa19a1,[10SEJ],S2B_MSIL2A_20240730T185919_R013_T10SEJ_2024073...
3,"POLYGON ((-2221113.743 2056469.316, -2216113.7...",4287d16f-a641-4392-80bd-aad968f74125,[10SEJ],S2B_MSIL2A_20240730T185919_R013_T10SEJ_2024073...
4,"POLYGON ((-2216113.743 2056469.316, -2211113.7...",b2ece92e-06ca-4ae8-b612-9d72f3e05dbe,[10SEJ],S2B_MSIL2A_20240730T185919_R013_T10SEJ_2024073...
...,...,...,...,...
1515,"POLYGON ((-2306113.743 2291469.316, -2301113.7...",97ac8454-3728-4af4-a7b8-30e6cbfe881c,[10TDK],S2A_MSIL2A_20240705T185921_R013_T10TDK_2024070...
1516,"POLYGON ((-2301113.743 2291469.316, -2296113.7...",f951fb10-5825-4f51-9c84-586324d7cdf8,[10TDK],S2A_MSIL2A_20240705T185921_R013_T10TDK_2024070...
1517,"POLYGON ((-2326113.743 2296469.316, -2321113.7...",e8c0536a-597c-4c6d-9554-0314d425e35b,[10TDK],S2A_MSIL2A_20240705T185921_R013_T10TDK_2024070...
1518,"POLYGON ((-2321113.743 2296469.316, -2316113.7...",76d83ac8-2b32-479e-a511-fae8d9dc78fb,[10TDK],S2A_MSIL2A_20240705T185921_R013_T10TDK_2024070...


In [27]:
m_best_results = leafmap.Map(center=[39.7, -123], zoom=8)
m_best_results.add_gdf(s2_grid_subset, layer_name='s2_tiles', style={"color": "red"})
m_best_results.add_gdf(best_results, layer_name='vector_tiles_s2_grid', style={"color": "green"})
m_best_results

Map(center=[39.7, -123], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title', 'zoom_out…

In [28]:
group_by_product = grouped_gdf = best_results.groupby("best_s2_product_id")["feature_id"].agg(list).reset_index()
group_by_product

,best_s2_product_id,feature_id
0,S2A_MSIL2A_20220723T184931_R113_T10TEK_2022072...,"[77eef212-9f9c-4817-b540-26331b75f68f, f5d2535..."
1,S2A_MSIL2A_20240705T185921_R013_T10TDK_2024070...,"[8e302035-4f2e-4966-8c3c-2e0ac49dffe0, f2fa733..."
2,S2A_MSIL2A_20240712T184931_R113_T10SDJ_2024071...,"[54e11c03-f37e-43d5-bc25-dde4079e2f8a, e4cb502..."
3,S2B_MSIL2A_20240730T185919_R013_T10SEJ_2024073...,"[76f33694-bbf2-4def-b1dc-6a891fc9877c, fc04cdf..."


In [ ]:
def download_merge_and_clip(stac_client: StacSearch, product_id):
    query={"eo:product_id": {"eq": product_id}},  # Adjust the query to match the correct field
    collections=["sentinel-2-l1c"]
    bands = bands = ["B02", "B03", "B04", "B08", "visual"]
    stac_client.seach(collections=collections, query=query)
    assets = stac_client.download_search_results(bands=bands, base_directory= DATA_DIR)
    for asset in assets:
        asset.merge_asset(base_directory=DATA_DIR)
        asset.reproject_merged_asset(target_projection=5070)

## Exploring the Sentinel 2 STAC catalog tools

In [29]:
# from pathlib import Path
# from geospatial_tools import DATA_DIR
# from geospatial_tools.stac import StacSearch, PLANETARY_COMPUTER
# from geospatial_tools.utils import create_date_range_for_specific_period

In [30]:
# start_year = 2023
# end_year = 2024
# start_month = 6
# end_month = 7
# date_ranges = create_date_range_for_specific_period(start_year=start_year, end_year=end_year,
#                                                     start_month_range=start_month, end_month_range=end_month)
# 
# search_client = StacSearch(PLANETARY_COMPUTER)
# 
# collection = "sentinel-2-l2a"
# tile_ids = ["10SDJ"]
# query = {"eo:cloud_cover": {"lt": 1}, "s2:mgrs_tile": {"in": tile_ids}}
# sortby = [{"field": "properties.eo:cloud_cover", "direction": "asc"}]
# 
# results = search_client.stac_api_search_for_date_ranges(date_ranges=date_ranges, collections=collection, query=query,
#                                                         sortby=sortby, max_items=20)
# 
# sorted_items = search_client.sort_results_by_cloud_coverage()
# optimal_result = sorted_items[0]
# 
# for item in sorted_items:
#     print(f"Item: {item.id}, {item.datetime}, {item.properties['eo:cloud_cover']}")
# print(f"Optimal result: {optimal_result.id}, {optimal_result.datetime}, {optimal_result.properties['eo:cloud_cover']}")

In [31]:
# bands = ["B02", "B03", "B04", "B08", "visual"]
# file_base_path = Path(f"{DATA_DIR}/sentinel-2/")
# best_result = search_client.download_best_cloud_cover_results(bands=bands, base_directory=file_base_path)
# best_result

In [32]:
# best_result.show_asset_items()

In [33]:
# merged = best_result.merge_asset(delete_sub_items=True)
# merged

In [34]:
# reprojected = best_result.reproject_merged_asset(target_projection=5070, delete_merged_asset=True)
# reprojected